# SiumAI Quickstart

Welcome to the SiumAI quickstart guide, your first step in exploring the world of application development with agent-based large language models (or in short, agentic llm).

XEntropy is an innovative development platform designed to elevate the ease at which developers can construct intelligent applications. Throughout this tutorial, we will navigate the following key areas:

1. **Account creation:** We guide you step-by-step in establishing your XEntropy account:the gateway to a host of agentic llm tools.

2. **Loading tools from XEntropy:** Learn how to seamlessly and efficiently import tools from the platform into your environment to kickstart your foray into developing llm agents and intelligent applications.

3. **Managing your account:** Your XEntropy account is useful for many things. Learn how to:
    - **Purchase credit**: Convenience and accessibility are core to XEntropy. Understand how to replenish your account balance in a straightforward, user-friendly manner.

   - **Get paid**: Having created applications or tools that other developers find useful? Here's how to benefit from your creations.

   - **Modify and delete tools**: As your skill level advances, so might your tools and applications. Learn how to adjust and remove tools from your portfolio.

   - **Utilisation of logs**: Track your progress and rectify mistakes by understanding how to make use of logs effectively.

   - **Finetune your own agentic llm**: Capture the essence of your application development ideation by discovering how to fine-tune your personal agentic llm to reach peak performance.

## Installation

In [ ]:
# Install AgentX
from distutils.dir_util import copy_tree, remove_tree
import os
import sys

site_packages_path = [x for x in sys.path if x.endswith('site-packages')][0]
# remove any existing installation
if os.path.exists(f'{site_packages_path}/siumai'):
    remove_tree(f'{site_packages_path}/siumai')
copy_tree(f'../siumai', f'{site_packages_path}/siumai')

In [ ]:
# Install dependencies to complete this tutorial
%pip install -r requirements.txt

## Loading tools from XEntropy

Agentic LLM is valuable in deducing, predicting, and problem-solving tasks where results need to be accomplished through sequential midsteps. When applying this pattern, you need to consider the following:

1. Define a clear Objective: This is the end goal that the agent wants to achieve. Explicitly specifying the objective helps guide the agent throughout the process.

2. Break down the Problem: Define the context of the problem in chunks that the model can handle within its cognitive limit. This will facilitate better understanding and allow the model to take it one step at a time. You can create an agent for each chunk to faciliate the process. Refer to `composite-agents-tool` for more information.

3. Formulate Actionable Steps: The agent needs to have means or steps to achieve the objective. Search on XEntropy to see if you can utilize existing tools to help with the process. If not, you can create your own tools to help with the process.

4. Monitor Progress: Track the progress of the model in each step to make sure it is on track and making progress towards the objective. If not, adjustments might be necessary.

Addressing Hallucination: One of the challenges with LLMs is that they may end up creating information that is not accurate, also known as hallucinating. This can be managed in the following ways:

1. Utilize Verifiable Sources: Whenever possible, try feeding the model with prompt that encourages the use of verifiable sources or requires the provision of source-based information. This increases the probability of information accuracy. You may refer to `rag-tool` and `api-tool` for more information.

2. Use Continual Feedback Loop: Applying a feedback loop between the user and the model’s output also helps in curbing hallucination issues. If the model’s output is incorrect, it can be traced back, analyzed, and altered.

3. External Calculation and Validation: For tasks that require high preciseness and accuracy, external computations should be conducted. Also, results generated by the model should be cross-verified with facts. You may refer to `logic-tool` for more information.


You can import specific tools from XEntropy, which are designed to provide updated and accurate information, alleviate model hallucination, and help with external calculations. These can be easily integrated into your existing applications and designed workflows to foster effective problem-solving using agentic LLM.

In [ ]:
with open('.env', 'w') as f:
    f.write('\n'.join(
        [
            f'{key}={value}' for key, value in {
                'AZURE_OPENAI_KEY': 'YOUR_AZURE_OPENAI_KEY', # replace with your own Azure OpenAI key
                'AZURE_OPENAI_ENDPOINT': 'YOUR_AZURE_OPENAI_ENDPOINT', # replace with your own Azure OpenAI endpoint
                # 'OPENAI_API_KEY': 'YOUR OPENAI_API_KEY', # replace with your own OpenAI key
                'XENTROPY_API_KEY': "YOUR_XENTROPY_API_KEY", # replace with your own Xentropy key
            }.items()
        ]
    ))

In [ ]:
# load environment variables
from dotenv import load_dotenv

load_dotenv()

### XEntropy Marketplace
XEntropy maintains a marketplace for tools. You can search suitable tools to complete an objective.

In [ ]:
from siumai.tool import Tool
import os

tool_search = Tool.load(
    'xentropy:tool_search', 
    api_key=os.environ.get('XENTROPY_API_KEY')
)

# display the function call schema
tool_search.input_json_schema

In [ ]:
from rich import print as rich_print
import json
# run the tool
rich_print(
    json.loads(
        tool_search.run(query='convert an address to geolocation')
    )
)

In [ ]:
# Get a tool that converts address to latitude longitude coordinate
geocoding = Tool.load('xentropy--geocoding', api_key=os.environ.get('XENTROPY_API_KEY'))
# Get a tool that computes the earth surface distance between two coordinates
geodesic = Tool.load('xentropy--geodesic', api_key=os.environ.get('XENTROPY_API_KEY'))

## Creating Agents with the loaded tools

In [ ]:
from siumai.agent import Agent
from siumai.schema import GenerationConfig, Message, Content
from typing import List

tools = [geocoding, geodesic]

generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
    azure_deployment='gpt-35',
)


# replace with your own azure deployment to choose different models
generation_config.model = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

# define a termination function
# once the agent execute the geodesic tool, the conversation ends
def terminate(history:List[Message]):
    tool_response_name = [
        message.content.tool_response.name for message in history if message.content.tool_response
    ]
    if 'xentropy--geodesic' in tool_response_name:
        return True
    return False
    
# define the agent
agent = Agent(
    name='agent',
    generation_config=generation_config,
    system_prompt='Use the functions you have been provided to solve the problem. Reply TERMINATE to end the conversation when the problem is solved.',
    tools = tools,
    termination_function=terminate,
)

In [ ]:
# the agent is ready to generate messages
messages = [
    Message(
        role='user',
        content=Content(
            text='What is the distance between Gare Port La Goulette - Sud in Tunisia and Porto di Napoli in Italy?',
        ),
    )
]

In [ ]:
# use the agent to use geocoding and geodesic to find earth surface distance between two locations

max_iterations = 10

for i in range(max_iterations):
    response = agent.generate_response(messages)
    # termination condition is met
    if response == None:
        break
    else:
        [rich_print(r.model_dump(exclude_unset=True)) for r in response]
    messages += response

### Share and get paid for your tools
You can checkout the following examples to see how to create and share tools on XEntropy.
- [api](https://github.com/ckh112/xentropy/blob/main/docs/api-tool/notebook.ipynb)
- [agent](https://github.com/ckh112/xentropy/blob/main/docs/composite-agents-tool/notebook.ipynb)
- [rag](https://github.com/ckh112/xentropy/blob/main/docs/rag-tool/notebook.ipynb)

## Managing your account

The `Client` class has a few other utilities to access XEntropy platform.

In [ ]:
from siumai.client import Client
client = Client(api_key=os.environ.get('XENTROPY_API_KEY'))

# See your account summary and usage data on each tool
client.summary()

In [ ]:
# XEntropy credit can be used to pay for tool usage, if the tool is not free.
# You can top up your XEntropy credit by paying with credit card by navigating to the payment link.
print(client.summary().get('payment_link'))

In [ ]:
# Alternatively you can also top up your XEntropy credit by paying with cryptocurrency.
# We accept USDT, USDC, and DAI on the ethereum blockchain.
# First, register your address.
from eth_account import Account
account = Account.from_key('YOUR_PRIVATE_KEY')
# you can also use mnemonic phrase
# account = Account.from_mnemonic('YOUR_MNEMONIC_PHRASE')
response = client.register_ethereum_address(account=account)
print(response.json())

In [ ]:
# Sometimes you'd like to modify your published tool
# Example 1: Making the tool public to XEntropy platform user
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="public",
    value=True,
)

In [ ]:
# Example 2: Tool description acts as a prompt to LLM, and hence it can be optimised through prompt engineering.
# You may want to modify it for improved performance.
# Check out https://github.com/microsoft/LMOps/tree/main/prompt_optimization for inspiration.
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="description",
    value="YOU_NEW_DESCRIPTION",
)

In [ ]:
# You can take your tool away anytime.
client.delete_tool(
    tool="YOUR_TOOL_NAME"
)

XEntropy pays 80% of the tool-use revenue to tool developers's `payout` wallet. There is also a 5% bonus paid to the `balance` wallet to incentivize tool developers for consuming other tools on XEntropy.

At this moment we only support withdrawl on the ethereum network for selected stable coins. More withdrawal method is on the way.

In [ ]:
# To withdraw from your payout wallet
client.stable_coin_payout(
    amount=100000,  # amount to withdraw denominated in XEntropy Credit. i.e. 100000 XEntropy Credit = 1 USD
    address="YOUR_ETHEREUM_ADDRESS", # your ethereum network address
    stable_coin='USDT' # choose between ['USDT', 'USDC', 'DAI']
)